# Seasonal Effects on Female Red-Billed Gull Weight in New Zealand.
**Prepared by Group 4: Arnab Das, Maggie Wang, Paul Zeng, Dylan Zhang**

## Introduction

## Methods and Results

## Discussion

## References